# Basic Brick Stack with Mortar Computation

This notebook demonstrates how to create two stacked bricks and compute the mortar volume between them using boolean intersection operations.

## Setup: Import Libraries and Define Parameters

Import COMPAS geometry libraries and define brick dimensions and mortar parameters.

In [ ]:
import compas.geometry as cg
import compas.datastructures as cd
from compas_notebook.viewer import Viewer
# Create a viewer
viewer = Viewer()

BRICK_DIM = [0.230, 0.110, 0.050]
MORTAR_THICKNESS = 0.010
LAYER_HEIGHT = BRICK_DIM[2] + MORTAR_THICKNESS
MAXIMUM_INTERSECTION_DISTANCE = (BRICK_DIM[0]**2 + BRICK_DIM[1]**2 + BRICK_DIM[2]**2) ** 0.5 - 0.01

## Create Two Stacked Bricks

Define coordinate frames and create two brick boxes with slight offset to simulate real-world stacking.

In [ ]:
# Define two stacked brickes
frame1 = cg.Frame([0, 0, 0], [1, 0, 0], [0, 1, 0])
frame2 = cg.Frame([0, 0.01, LAYER_HEIGHT], [1, 1, 0], [0, 1, 0])

brick_1 = cg.Box(BRICK_DIM[0], BRICK_DIM[1], BRICK_DIM[2], frame1)
brick_2 = cg.Box(BRICK_DIM[0], BRICK_DIM[1], BRICK_DIM[2], frame2)

viewer.scene.add(brick_1, color=(230, 230, 150))
viewer.scene.add(brick_2, color=(150, 230, 230))
viewer.show()

## Define Mortar Computation Function

Create a function to compute the mortar volume between two bricks using boolean intersection of enlarged brick volumes.

In [ ]:
def compute_mortar(brick_1, brick_2):
    """
    Compute the mortar between two brickes and return as a mesh.
    """

    # instersection is computational expensive, so we comute it only when brickes are close enough
    distance = brick_1.frame.point.distance_to_point(brick_2.frame.point)
    if distance > MAXIMUM_INTERSECTION_DISTANCE:
        return None


    # generate larger brick for boolean intersection
    brick_1_large = cg.Box(
        BRICK_DIM[0],
        BRICK_DIM[1],
        BRICK_DIM[2] + MORTAR_THICKNESS*2,
        brick_1.frame
    )
    brick_2_large = cg.Box(
        BRICK_DIM[0],
        BRICK_DIM[1],
        BRICK_DIM[2] + MORTAR_THICKNESS*2,
        brick_2.frame
    )

    # boolean intersection takes vertices and faces as input
    brick_1_vf = brick_1_large.to_mesh(True, 10, 10).to_vertices_and_faces()
    brick_2_vf = brick_2_large.to_mesh(True, 10, 10).to_vertices_and_faces()

    # perform boolean intersection
    intersection = cg.boolean_intersection_mesh_mesh(brick_1_vf, brick_2_vf)

    # convert result to mesh
    intersection_mesh = cd.Mesh.from_vertices_and_faces(*intersection)
    
    return intersection_mesh
    

## Compute and Visualize Mortar

Apply the mortar computation function and add the resulting mortar volume to the 3D viewer in gray.

In [ ]:
mortar = compute_mortar(brick_1, brick_2)
if mortar:
    viewer.scene.add(mortar, color=(200, 200, 200), opacity=0.6)
viewer.show()